<style>
@import url(https://www.numfys.net/static/css/nbstyle.css);
</style>
<a href="https://www.numfys.net"><img class="logo" /></a>
# Модель эволюции Бака-Снеппена

### Examples - Biophysics
<section class="post-meta">
By Thorvald M. Ballestad, Niels Henrik Aase, Jenny Lunde, Sondre Lundemo, and Jon Andreas Støvneng.
</section>
Last edited: October 6th 2020.

---

Этот блокнот несколько отличается от нашего обычного содержимого.
Рассматриваемая проблема и цифры довольно просты.
Цель блокнота - продемонстрировать язык Джулия, язык, который появился в последние несколько лет.
Джулия в какой-то степени уже закрепилась в научном сообществе и, скорее всего, продолжит расширяться в ближайшие годы.
Несмотря на множество чудес Python, самая большая проблема заключается в том, что он может быть медленным.
Использование NumPy происходит чрезвычайно быстро, так как за кулисами он представляет собой высоко оптимизированный скомпилированный код и использует преимущества таких известных библиотек, как BLAS и LAPACK.
Однако иногда использование NumPy может быть очень громоздким или невозможным, и приходится использовать циклы Python.
В этих сценариях другие языки могут быть более подходящими.
Многие сообщества, как в академических кругах, так и в промышленности, используют C++ из-за его скорости и мощных функций, хотя его разработка может быть как утомительной, так и просто слишком сложной.
Кроме того, без должных знаний можно было бы писать медленный код и на C++.
Фортран также является популярной альтернативой, особенно в научном сообществе.
Поскольку первая версия была выпущена более 60 лет назад, это один из старых языков, все еще широко используемых в вычислительной физике.
Он ценится за свою скорость, но может напугать и сбить с толку новых разработчиков.
Гибридные варианты, такие как использование Фортрана только для внутреннего цикла, обертывание его в Python, являются одним из возможных методов, о котором вы можете прочитать подробнее [здесь](https://nbviewer.jupyter.org/urls/www.numfys.net/media/notebooks/fortran_to_python.ipynb) [[1]](#f2py) и [здесь](https://nbviewer.jupyter.org/urls/www.numfys.net/media/notebooks/Monte_Carlo_Ising.ipynb) [[2]](#monte_carlo).

Джулия - довольно молодой язык, с огромным потенциалом.
Она направлена на то, чтобы занять промежуточное положение между Python, с одной стороны, и Fortran и C++, с другой.
На этом мы реализуем модель эволюции Бака-Снеппена в Джулии.

Большая часть кода здесь будет легко понята тем, кто привык к Python.
Мы находимся в процессе написания вступительной тетради для Джулии, которую мы надеемся скоро закончить, и с которой мы свяжемся здесь, когда закончим.

## Модель

Модель Бака-Снеппена[[3]](#bak-sneppen-paper) проста.
Она дает нам некоторое базовое представление о процессе эволюции.
У нас есть $N$ видов.
Каждому виду приписывается коэффициент пригодности, случайное число от 0 до 1.
Виды распределены по кольцу, т. е. по линии с периодическими граничными условиями.
Идея модели состоит в том, чтобы применить к этой системе только очень простой набор правил и посмотреть, как она развивается.
Моделирование состоит из двух этапов:
 1. Найдите самый слабый вид, тот, у которого самый низкий коэффициент приспособленности.
 2. Дайте этому виду и его двум соседям новые случайные факторы пригодности.
 
На первый взгляд эти два правила могут показаться произвольными и немотивированными, но учтите следующее:
Для подходящих видов изменение их фактора приспособленности (мутация) в среднем будет оказывать негативное влияние.
Мутировавшее потомство подходящего вида, скорее всего, не принесет пользы этому виду и, таким образом, умрет.
Однако менее приспособленные виды находятся под угрозой вымирания.
Их мутировавшее потомство будет иметь больше шансов на выживание, чем их родители; мы позволяем первоначальному виду вымереть, а их мутировавшему потомству выжить.
Мы назначаем новый (случайный) коэффициент пригодности мутировавшему потомству.
Это мотивирует правило 1 (и первую часть правила 2).
Виды в экосистеме не изолированы.
Возмущения в какой-то части экосистемы могут повлиять на всю экосистему, даже на виды, которые вообще не взаимодействуют напрямую.
Здесь мы ограничим взаимодействие ближайшими соседями - мы игнорируем косвенные эффекты.
Представьте, что определенная порода кроликов в какой-то момент мутирует и становится меньше и быстрее.
Для серой лисы и рыси, охотящихся на кролика, это угрожает их доступу к пище - кролик слишком быстр и проворен, чтобы его поймать.
Однако маленькая серая лиса обнаруживает, что она достаточно мала, чтобы копаться в кроличьей норе, и поэтому в основном не подвержена изменениям.
Рыси, однако, не так повезло.
Она слишком велика, чтобы поместиться в кроличьей норе, и изо всех сил пытается найти достаточно пищи.
Этот пример показывает, как мутация одного вида может по-разному повлиять на соседние виды.
В нашей модели мы учитываем это, изменяя коэффициент пригодности двух ближайших соседей в экосистеме.
Значение коэффициента пригодности двух ближайших соседей присваивается случайным образом, что означает, что эффект их мутировавшего соседа может быть как отрицательным, так и положительным.
Это мотивирует правило 2.

## Реализация

Задача просто состоит в циклическом изменении вида.
Этот тип проблем, итеративные шаги, где каждый шаг зависит от предыдущего, как правило, трудно реализовать с помощью NumPy, по крайней мере напрямую.
Поскольку в конечном итоге мы напишем некоторую циклическую структуру, быстрый язык будет предпочтительнее Python'а - в нашем случае это Джулия.
Сравнивая показанную здесь реализацию с аналогичной реализацией в Python, мы измерили, что реализация Julia примерно в два раза быстрее, чем в Python.


Единственной программной проблемой, требующей решения, являются периодические граничные условия.
Наш вид будет представлен в виде одномерного вектора с числами от 0 до 1, коэффициентами пригодности.
Граничные условия играют роль только при нахождении соседей.
Наивный, но функциональный подход заключается в том, чтобы для каждого шага проверять, находится ли он на границе, а затем выполнять специальный код для этих случаев.
Однако мы используем общую технику, вдохновленную теорией графов.
Каждый элемент вектора, узел $i$, имеет двух соседей, одного правого и одного левого, обычно узел $i+1$ и $i-1$.
Мы создадим два вектора, один с правыми индексами и один с левыми индексами.
Элемент в позиции $i$ в нашем правом индексном векторе будет иметь значение $i+1$, за исключением границы.
Аналогично для указателя слева.
Это позволяет писать очень чистый код. [<sup>1</sup>](#footnote-1)

<center>
<img src="images/index_fig.png" alt="Index arrays" /><br />
</center>

На рисунке показаны массивы `ir` и `il`, которые при индексе `i` дают индекс правого и левого элемента.

По мере того как моделирование повторяется во времени, мы будем для каждой итерации сохранять минимальный коэффициент пригодности в этой итерации, а также возраст каждого вида. Мы можем интерпретировать возраст как годы, прошедшие с момента закрепления мутации.
Возраст рассчитывается путем увеличения возраста на единицу для каждого раунда и установки его на ноль, когда вид "мутирует". 

In [ ]:
N = 1000  # Number of species.
t_end = 100000  # End time of simulation.
t_space = 1:t_end  # An iterable for the time steps. Note that Julia is 1-indexed.

# Create the species' fitness factor.
# rand(N) returns a vector of size N with random elements between 0.0 and 1.0.
species = rand(N)

# Fix boundary conditions
# ir and il are right indices and left indices respectively
# ir[i] gives the right index at position i, ususally i+1.
#
# `collect` converts a range, for example 2:N, into an array.
# It serves the same purpose as doing `list(range(2, N))` in Python.
ir = collect(2:N+1)
il = collect(0:N-1)
ir[N] = 1  # Right of rightmost is leftmost in a ring
il[1] = N  # Left of leftmost is rightmost

"""
Simulate the species over the given time interval.
Return the age of each species at each time step and 
the minimum fitness factor at each time step.
"""
function simulate!(species, t_space)
    age = zeros(N, t_end)
    mins = zeros(t_end)
    
    for t in t_space[2:end]
        age[:, t] = age[:, t-1] .+ 1
        # We want to find the weakest species.
        # `findmin` returns the minimum value and the index of that value.
        mins[t], index_min = findmin(species)
        species[index_min] = rand()
        species[ir[index_min]] = rand()  # Right
        species[il[index_min]] = rand()  # Left
        
        age[index_min, t] = 0
        age[ir[index_min], t] = 0  # Right
        age[il[index_min], t] = 0  # Left
    end
    
    return age, mins
end

In [ ]:
@time age, mins = simulate!(species, t_space);
# We add a semicolon so that Jupter does not print the values of age and mins.

Джулия поддерживает несколько бэкендов для построения графиков.
Здесь мы будем использовать PyPlot, который является просто интерфейсом к Matplotlib Python.
Для любого, кто знаком с Matplotlib, следующий код должен быть легко понятен.

In [ ]:
using PyPlot
# There is a lot happening in this first line. Let's break it down.
# The `[:, 1:100:end]` resembles Python's slicing syntax, but there are slight differences.
# The first colon simply means every row, just like Python.
# `1:100:end` follow the syntax `START:STEP:END`, so it means
# 'from first column to last column, take every 100th column'.
# `transpose` obviously transposes the matrix, this is done only
# to have the the axis where we think they make the most sense,
# ie. time on second axis.
# `vmax` is the same as in matplotlib. It is included to
# give better contrast in the figure, by setting all ages over
# `vmax` to the same color, bringing out the details in
# rapidly changing areas.
plt.imshow(transpose(age[:, 1:100:end]), vmax=30000)
plt.xlabel("Species")
plt.ylabel("Age [# iterations]")
plt.title("The age of species (x-axis) through the simulation.")
plt.show()

In [ ]:
using Plots
# There is a lot happening in this first line. Let's break it down.
# The `[:, 1:100:end]` resembles Python's slicing syntax, but there are slight differences.
# The first colon simply means every row, just like Python.
# `1:100:end` follow the syntax `START:STEP:END`, so it means
# 'from first column to last column, take every 100th column'.
# `transpose` obviously transposes the matrix, this is done only
# to have the the axis where we think they make the most sense,
# ie. time on second axis.
# `vmax` is the same as in matplotlib. It is included to
# give better contrast in the figure, by setting all ages over
# `vmax` to the same color, bringing out the details in
# rapidly changing areas.

heatmap(transpose(age[:, 1:100:end]), vmax=30000)
xaxis!("Species")
yaxis!("Age [# iterations]")
title!("The age of species (x-axis) through the simulation.")

На рисунке выше показан возраст вида с течением времени.
Обратите внимание на группировку видов, где пространственно близкие виды, как правило, выживают вместе в течение длительного времени.
Можно рассматривать кластеры как устойчивые подэкосистемы. Эволюции, как правило, локализуются как во времени, так и в пространстве.
Если важный вид в стабильной подэкосистеме внезапно мутирует, эффект этой единственной мутации распространяется по всей подэкосистеме, приводя к мутации у каждого вида, населяющего эту подэкосистему.
Эта последовательность массовых эволюций называется лавинами, явление, которое много изучается в области статистической физики.  

Теперь мы рассмотрим минимальный коэффициент пригодности в экосистеме как функцию времени.
Мы видим, что он быстро увеличивается, а затем сужается до асимптотически стабильного значения, примерно равного 0.6.
Это объясняет локализацию эволюций, которые мы видим на возрастном графике.
Когда происходит мутация, вероятность того, что три новых сгенерированных значения будут ниже любого другого значения, довольно высока, по крайней мере, через некоторое время, когда минимальное значение будет высоким.

Теперь, по общему признанию, следующий код довольно плотный.
Можно было бы написать этот фрагмент таким образом, чтобы он был более понятен тем, кто не знаком с Джулией.
Однако это решение короче, эффективнее, и в конечном счете мы надеемся, что оно может передать часть мощи Джулии.
Итак, что происходит в этой единственной строке кода?
Мы хотим посмотреть, как минимальные значения изменяются с течением времени.
Наивный подход к простому построению значений `mins`, к сожалению, ничего не даст, так как слишком много статистического шума.
Мы будем придерживаться более утонченного подхода.
Разделим ось времени и возьмите максимальное значение `mins` в каждом разделе.
Или, говоря по-другому, сгруппируем значения `mins` в "корзины" по N значений в каждой и возьмем максимум в каждой корзине.

Затем построим эти значения.
"Утилиты итерации" Джулии предлагают итератор `Iterators.partition`.
Она делает именно то, что мы только что описали, при заданном некотором векторе, делит его на корзины, каждая из которых содержит некоторое заданное количество элементов.
Таким образом, `Iterators.partition(mins, 500)` возвращает итератор.
Каждый элемент в итераторе представляет собой вектор с 500 элементами, взятыми из `mins`.
Затем мы просто берем максимальное значение этого вектора.
Было сделано обоснованное предположение, чтобы прийти к цифре 500.
Само по себе число не несет никакого физического значения;
нам нужно какое-то значение, достаточно большое, чтобы отфильтровать шум, но достаточно малое, чтобы не удалять интересующие нас данные.

In [ ]:
basket_size = 500
plot([maximum(a) for a in Iterators.partition(mins, basket_size)])
xaxis!("Minimum fitness factor")
yaxis!("Time")
# Since we collected our results in baskets, the numbers on the time-axis are wrong.
# We either have to rescale them or remove them.
# Since we are mainly interested in the qualitative behavior, we remove the numbers.

Теперь мы признаем, что и модель, и анализ модели в этом блокноте намного проще, чем то, что мы обычно делаем в наших блокнотах.
Однако цель этой записной книжки состоит в том, чтобы просто проиллюстрировать, что Джулия является жизнеспособным вариантом для написания современных эффективных вычислений.
Мы надеемся, что вы нашли эту записную книжку несколько вдохновляющей и, надеюсь, она может послужить добрым знакомством с Джулией.

## Сноски
<span id="footnote-1"><sup>1</span> Не так просто предсказать, приведет ли это к более быстрому времени выполнения или нет. В данном конкретном примере это на самом деле медленнее, чем использование условного подхода, т.е. выполнение специального кода внутри оператора `if`, если он находится на границе. Это было обнаружено простым внедрением обоих методов и измерением времени выполнения. В общем, предсказать, что быстрее, сложно, и если эффективность важна, рекомендуется экспериментировать с различными реализациями. Однако метод, проиллюстрированный здесь, предлагает гораздо более чистый код, что становится все более важным в более сложных задачах.</span>

# References
 - <a name="f2py">[1]</a> https://nbviewer.jupyter.org/urls/www.numfys.net/media/notebooks/fortran_to_python.ipynb
 - <a name="monte_carlo">[2]</a> https://nbviewer.jupyter.org/urls/www.numfys.net/media/notebooks/Monte_Carlo_Ising.ipynb
 - <a name="bak-sneppen-paper">[3]</a> Bak, Per & Sneppen, Kim. (1994). Bak, P. & Snepen, K. Punctuated equilibrium and criticality in a simple model of evolution. Physical review letters. 71. 4083-4086. 10.1103/PhysRevLett.71.4083. 